In [10]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import sys
import re
from glob import glob
import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
#  sys.path.append(f"/mnt/c/Git/go/kaggle/github/library/")
import utils
from utils import logger_func, mkdir_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

import eda
from utils import get_categorical_features, get_numeric_features, parallel_process
from feature_engineering import diff_feature, division_feature, product_feature, cnt_encoding, select_category_value_agg, exclude_feature, target_encoding
from tqdm import tqdm

#========================================================================
# Global Variable
sys.path.append(f"../py")
from info_home_credit import hcdr_key_cols
key, target, ignore_list = hcdr_key_cols()
#========================================================================

base = utils.read_df_pkl('../input/base0*').set_index(key)

100%|██████████| 3/3 [00:00<00:00, 553.36it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
app = utils.read_df_pkl(path='../input/clean_app*.p')
display(app.columns)


100%|██████████| 3/3 [00:01<00:00,  2.00it/s]


Index(['AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_YEAR',
       ...
       'TARGET', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE',
       'WEEKDAY_APPR_PROCESS_START', 'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_MODE',
       'YEARS_BUILD_AVG', 'YEARS_BUILD_MEDI', 'YEARS_BUILD_MODE'],
      dtype='object', length=122)

In [45]:
num_list = get_numeric_features(app, ignore_list)
amt_list = [col for col in app.columns if col.count('AMT_') and not(col.count('BUREAU'))]
day_list = [col for col in app.columns if col.count('DAYS_')]
other_list = list(set(num_list) - set(amt_list) - set(day_list))

In [60]:
from new_FE import FeaturePack

region_list = [col for col in app.columns if col[:6]=='REGION']
ext_list = [col for col in app.columns if col.count('EXT_')]
fpack = FeaturePack(base=base, data=app.copy(), ignore_list=ignore_list, key=key, target=target)

In [47]:
prefix = 'f004_app_'
combi_list = []
for f1 in amt_list+day_list:
    for f2 in ext_list:
        combi_list.append([f1, f2])
fpack.go_interact(combi_list=combi_list, prefix=prefix, is_div=True, is_diff=False, is_pro=False)
sys.exit()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,FONDKAPREMONT_MODE,HOUR_APPR_PROCESS_START,HOUSETYPE_MODE,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_INCOME_TYPE,NAME_TYPE_SUITE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE,ORGANIZATION_TYPE,OWN_CAR_AGE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,SK_ID_CURR,TARGET,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,AMT_ANNUITY_div_EXT_SOURCE_1,AMT_ANNUITY_div_EXT_SOURCE_2,AMT_ANNUITY_div_EXT_SOURCE_3,AMT_CREDIT_div_EXT_SOURCE_1,AMT_CREDIT_div_EXT_SOURCE_2,AMT_CREDIT_div_EXT_SOURCE_3,AMT_GOODS_PRICE_div_EXT_SOURCE_1,AMT_GOODS_PRICE_div_EXT_SOURCE_2,AMT_GOODS_PRICE_div_EXT_SOURCE_3,AMT_INCOME_TOTAL_div_EXT_SOURCE_1,AMT_INCOME_TOTAL_div_EXT_SOURCE_2,AMT_INCOME_TOTAL_div_EXT_SOURCE_3,DAYS_BIRTH_div_EXT_SOURCE_1,DAYS_BIRTH_div_EXT_SOURCE_2,DAYS_BIRTH_div_EXT_SOURCE_3,DAYS_EMPLOYED_div_EXT_SOURCE_1,DAYS_EMPLOYED_div_EXT_SOURCE_2,DAYS_EMPLOYED_div_EXT_SOURCE_3,DAYS_ID_PUBLISH_div_EXT_SOURCE_1,DAYS_ID_PUBLISH_div_EXT_SOURCE_2,DAYS_ID_PUBLISH_div_EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE_div_EXT_SOURCE_1,DAYS_LAST_PHONE_CHANGE_div_EXT_SOURCE_2,DAYS_LAST_PHONE_CHANGE_div_EXT_SOURCE_3,DAYS_REGISTRATION_div_EXT_SOURCE_1,DAYS_REGISTRATION_div_EXT_SOURCE_2,DAYS_REGISTRATION_div_EXT_SOURCE_3
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0247,0.0250,0.0252,0.0369,0.0369,0.0383,0,1.0,M,0.0143,0.0144,0.0144,-9461,-637,-2120,-1134.0,-3648.0,2.0,2.0,0.00,0.00,0.0000,No,0.0690,0.0690,0.0690,9.000000e+08,9.000000e+08,9.000000e+08,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,N,Y,1,0,0.0833,0.0833,0.0833,0.1250,0.1250,0.1250,reg oper account,10,block of flats,0.0369,0.0375,0.0377,0,0,0.0202,0.0205,0.022,0.0190,0.0193,0.0198,Cash loans,Secondary / secondary special,Single / not married,House / apartment,Working,Unaccompanied,0.0000,0.0000,0.0,0.0000,0.00,0.0,2.0,2.0,Laborers,Business Entity Type 3,NaN,0.018801,2,2,0,0,0,0,100002,1.0,0.0149,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341,0.000247,0.000247,0.000247,0.002033,0.002033,0.002033,0.001170,0.001170,0.00117,0.000506,0.000506,0.000506,-0.000019,-0.000019,-0.000019,-0.000001,-1.061667e-06,-1.061667e-06,-3.028571e-0

In [61]:
prefix = 'f004_app_'
combi_list = []
for f1 in ext_list:
    for f2 in ext_list:
        if f1==f2:continue
        combi_list.append([f1, f2])
fpack.go_interact(combi_list=combi_list, prefix=prefix)
sys.exit()

[Calcurate Intaract] done in 0 s
(307511,) | EXT_SOURCE_1_diff_EXT_SOURCE_2
(307511,) | EXT_SOURCE_1_div_EXT_SOURCE_2
(307511,) | EXT_SOURCE_1_pro_EXT_SOURCE_2
(307511,) | EXT_SOURCE_1_diff_EXT_SOURCE_3
(307511,) | EXT_SOURCE_1_div_EXT_SOURCE_3
(307511,) | EXT_SOURCE_1_pro_EXT_SOURCE_3
(307511,) | EXT_SOURCE_2_diff_EXT_SOURCE_1
(307511,) | EXT_SOURCE_2_div_EXT_SOURCE_1
(307511,) | EXT_SOURCE_2_pro_EXT_SOURCE_1
(307511,) | EXT_SOURCE_2_diff_EXT_SOURCE_3
(307511,) | EXT_SOURCE_2_div_EXT_SOURCE_3
(307511,) | EXT_SOURCE_2_pro_EXT_SOURCE_3
(307511,) | EXT_SOURCE_3_diff_EXT_SOURCE_1
(307511,) | EXT_SOURCE_3_div_EXT_SOURCE_1
(307511,) | EXT_SOURCE_3_pro_EXT_SOURCE_1
(307511,) | EXT_SOURCE_3_diff_EXT_SOURCE_2
(307511,) | EXT_SOURCE_3_div_EXT_SOURCE_2
(307511,) | EXT_SOURCE_3_pro_EXT_SOURCE_2
(48744,) | EXT_SOURCE_1_diff_EXT_SOURCE_2
(48744,) | EXT_SOURCE_1_div_EXT_SOURCE_2
(48744,) | EXT_SOURCE_1_pro_EXT_SOURCE_2
(48744,) | EXT_SOURCE_1_diff_EXT_SOURCE_3
(48744,) | EXT_SOURCE_1_div_EXT_SOURCE_

SystemExit: 

In [49]:
prefix = 'f004_app_'
combi_list = []
for f1 in amt_list+day_list:
    for f2 in region_list:
        combi_list.append([f1, f2])
fpack.go_interact(combi_list=combi_list, prefix=prefix, is_div=True, is_diff=False, is_pro=False)

[Calcurate Intaract] done in 0 s
(307511,) | AMT_ANNUITY_div_REGION_POPULATION_RELATIVE
(307511,) | AMT_ANNUITY_div_REGION_RATING_CLIENT
(307511,) | AMT_ANNUITY_div_REGION_RATING_CLIENT_W_CITY
(307511,) | AMT_CREDIT_div_REGION_POPULATION_RELATIVE
(307511,) | AMT_CREDIT_div_REGION_RATING_CLIENT
(307511,) | AMT_CREDIT_div_REGION_RATING_CLIENT_W_CITY
(307511,) | AMT_GOODS_PRICE_div_REGION_POPULATION_RELATIVE
(307511,) | AMT_GOODS_PRICE_div_REGION_RATING_CLIENT
(307511,) | AMT_GOODS_PRICE_div_REGION_RATING_CLIENT_W_CITY
(307511,) | AMT_INCOME_TOTAL_div_REGION_POPULATION_RELATIVE
(307511,) | AMT_INCOME_TOTAL_div_REGION_RATING_CLIENT
(307511,) | AMT_INCOME_TOTAL_div_REGION_RATING_CLIENT_W_CITY
(307511,) | DAYS_BIRTH_div_REGION_POPULATION_RELATIVE
(307511,) | DAYS_BIRTH_div_REGION_RATING_CLIENT
(307511,) | DAYS_BIRTH_div_REGION_RATING_CLIENT_W_CITY
(307511,) | DAYS_EMPLOYED_div_REGION_POPULATION_RELATIVE
(307511,) | DAYS_EMPLOYED_div_REGION_RATING_CLIENT
(307511,) | DAYS_EMPLOYED_div_REGION_R

SystemExit: 